# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
# import dependencies
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Experiment, Dataset
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.data.dataset_factory import TabularDatasetFactory
#from azureml.train.hyperdrive import uniform, choice
from azureml.core import ScriptRunConfig
from azureml.core import Environment
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

import os
import shutil


## Initialize Workspace and Create an Azure ML experiment

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

AZML
RG-AZML
eastus
a748a023-896b-4719-8f8e-3d1ba62d6e35


In [3]:
experiment_name = 'ideb-regression'

experiment=Experiment(ws, experiment_name)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
datastore = ws.get_default_datastore()
path = "data/data.csv"
try:
    ideb_dataset = Dataset.get_by_name(ws, name="ideb_dataset")
except:
    datastore.upload('data', target_path='data')
    # Create TabularDataset & register in workspace
    ideb_dataset = Dataset.Tabular.from_delimited_files([(datastore, path)])
    ideb_dataset = ideb_dataset.register(
        ws, name="ideb_dataset", create_new_version=True,
        description="Dataset for ideb prediction"
    )

## Creating Compute

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "automl-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Jobrunning.................................................................................................................................................................................................................
Running


## Create an environment

In [31]:
%%writefile conda_dependencies.yml

dependencies:
  - python=3.6.2
  - scikit-learn
  - numpy
  - pip:
    - azureml-defaults

Overwriting conda_dependencies.yml


In [32]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [42]:

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(
                        evaluation_interval=2,
                        slack_factor=0.1
)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( 
    {
         '--max_depth': choice(range(1, 20)),
         '--min_samples_split': choice(2, 5, 10, 15, 100),
         '--min_samples_leaf': choice(1, 2, 5, 10)
    }
)

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(
                source_directory="./",
                entry_script="train.py",
                compute_target=cpu_cluster_name
)

hyperdrive_config = HyperDriveConfig(
                    primary_metric_name='Accuracy',
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=20,
                    max_concurrent_runs=4,
                    hyperparameter_sampling=param_sampling,
                    policy=early_termination_policy,
                    estimator=estimator
)


##  Submit Run

In [43]:
# start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [44]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_abf263bb-be1b-4409-b970-2e0fbd8071f9
Web View: https://ml.azure.com/experiments/ideb-regression/runs/HD_abf263bb-be1b-4409-b970-2e0fbd8071f9?wsid=/subscriptions/a748a023-896b-4719-8f8e-3d1ba62d6e35/resourcegroups/RG-AZML/workspaces/AZML

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-21T01:54:54.153439][API][INFO]Experiment created<END>\n""<START>[2021-01-21T01:54:54.919306][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n""<START>[2021-01-21T01:54:54.675556][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2021-01-21T01:54:55.6696800Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [39]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
print(hyperdrive_run.get_metrics())



Run(Experiment: ideb-regression,
Id: HD_877b179c-0d1d-4227-b16c-7a260ea645d7_4,
Type: azureml.scriptrun,
Status: Completed)
{'HD_877b179c-0d1d-4227-b16c-7a260ea645d7_0': {'max_depth:': 5, 'min_samples_split:': 100, 'min_samples_leaf:': 2, 'Accuracy': 0.9744990256628848}, 'HD_877b179c-0d1d-4227-b16c-7a260ea645d7_1': {'max_depth:': 8, 'min_samples_split:': 10, 'min_samples_leaf:': 2, 'Accuracy': 0.9960866313961795}, 'HD_877b179c-0d1d-4227-b16c-7a260ea645d7_10': {'max_depth:': 8, 'min_samples_split:': 5, 'min_samples_leaf:': 5, 'Accuracy': 0.9956856074648475}, 'HD_877b179c-0d1d-4227-b16c-7a260ea645d7_11': {'max_depth:': 8, 'min_samples_split:': 5, 'min_samples_leaf:': 2, 'Accuracy': 0.9965194493059963}, 'HD_877b179c-0d1d-4227-b16c-7a260ea645d7_12': {'max_depth:': 8, 'min_samples_split:': 5, 'min_samples_leaf:': 10, 'Accuracy': 0.9951815948771828}, 'HD_877b179c-0d1d-4227-b16c-7a260ea645d7_13': {'max_depth:': 8, 'min_samples_split:': 100, 'min_samples_leaf:': 5, 'Accuracy': 0.98789378416329

In [40]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics=best_run.get_metrics()
best_run_files=best_run.get_file_names()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run ID',best_run.id)
print('\n Metrics: ', best_run_metrics)
print('\n Parameters: ', parameter_values,sep='\n')
print('\nAccuracy of Best run',best_run_metrics['Accuracy'],sep='\n')
print('\nBest run file names',best_run_files,sep='\n')

Best Run ID HD_877b179c-0d1d-4227-b16c-7a260ea645d7_4

 Metrics:  {'max_depth:': 30, 'min_samples_split:': 10, 'min_samples_leaf:': 2, 'Accuracy': 0.9972316182016638}

 Parameters: 
['--max_depth', '30', '--min_samples_leaf', '2', '--min_samples_split', '10']

Accuracy of Best run
0.9972316182016638

Best run file names
['azureml-logs/55_azureml-execution-tvmps_959b442c5a0ffa1d4b1ab577b7180775caf83e201c43664cdc30520d8654fe8c_d.txt', 'azureml-logs/65_job_prep-tvmps_959b442c5a0ffa1d4b1ab577b7180775caf83e201c43664cdc30520d8654fe8c_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_959b442c5a0ffa1d4b1ab577b7180775caf83e201c43664cdc30520d8654fe8c_d.txt', 'logs/azureml/100_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_70046a83-f85c-483f-8b41-0915e1f8ae92.jsonl', 'logs/azureml/dataprep/python_span_l_70046a83-f85c-483f-8b41-0915e1f8ae92.jsonl', 'logs/azureml/job_

## Save the best model

In [41]:
#TODO: Save the best model
model = best_run.register_model(model_name='best-hyper-model', model_path='outputs/model.pkl')
print('Best model registered')
model


model.download(target_dir="outputs", exist_ok=True)
print('model downloaded')

Best model registered
model downloaded


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service